In [7]:
import pandas as pd
import re
import os

# === 1. Función para limpiar y convertir un CSV al formato PVWatts v8 ===
def procesar_csv(nombre_archivo):
    print(f"🔵 Procesando archivo: {nombre_archivo}")

    # === 1.1 Leer las primeras líneas para extraer metadatos ===
    with open(nombre_archivo, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    lat, lon, elev = None, None, None
    for line in lines:
        if 'Latitude' in line:
            lat = float(line.split(':')[1].strip())
        if 'Longitude' in line:
            lon = float(line.split(':')[1].strip())
        if 'Elevation' in line:
            elev = float(line.split(':')[1].strip())

    if None in (lat, lon, elev):
        print("❌ No se encontraron correctamente latitud, longitud o elevación en el archivo.")
        return

    # === 1.2 Estimar ubicación y zona horaria corregida ===
    def estimar_ubicacion_y_utc(lat, lon):
        if -40 <= lat <= -30 and -75 <= lon <= -70:
            return "Chile (zona centro-sur)", -4
        elif 55 <= lat <= 65 and 140 <= lon <= 160:
            return "Rusia (Kamchatka)", 12
        elif 25 <= lat <= 35 and 85 <= lon <= 95:
            return "China (Tíbet)", 8
        else:
            return "Ubicación desconocida", round(lon / 15)

    ubicacion, time_zone = estimar_ubicacion_y_utc(lat, lon)

    # === 1.3 Formatear el signo de UTC ===
    def formato_utc(tz):
        return f"UTC{tz:+d}"

    print(f"   ➡ Latitud: {lat}, Longitud: {lon}, Elevación: {elev} m, Zona horaria estimada: {formato_utc(time_zone)}")
    print(f"   ➡ Ubicación estimada: {ubicacion}")

    # === 1.4 Encontrar la línea donde empieza el encabezado de datos ===
    for i, line in enumerate(lines):
        if line.strip().startswith('time(UTC)'):
            header_idx = i
            break

    # === 1.5 Leer los datos desde el encabezado ===
    data = pd.read_csv(nombre_archivo, skiprows=header_idx, encoding='utf-8')

    # === 1.6 Renombrar columnas para formato PVWatts ===
    data = data.rename(columns={
        'time(UTC)': 'timestamp',
        'T2m': 'Tdry',
        'RH': 'RH',
        'G(h)': 'GHI',
        'Gb(n)': 'DNI',
        'Gd(h)': 'DHI',
        'WS10m': 'Wspd',
        'WD10m': 'Wdir',
        'SP': 'Pres'
    })

    # === 1.7 Extraer Year, Month, Day, Hour, Minute del timestamp ===
    def parse_timestamp(ts):
        m = re.match(r'(\d{4})(\d{2})(\d{2}):(\d{2})(\d{2})', str(ts))
        if m:
            return int(m.group(1)), int(m.group(2)), int(m.group(3)), int(m.group(4)), int(m.group(5))
        else:
            return [None]*5

    data[['Year', 'Month', 'Day', 'Hour', 'Minute']] = data['timestamp'].apply(lambda x: pd.Series(parse_timestamp(x)))

    # === 1.8 Crear columnas requeridas ===
    data['Tdew'] = pd.NA
    data['Snow Depth'] = 0

    # === 1.9 Agregar latitud, longitud y zona horaria ===
    data['latitude'] = lat
    data['longitude'] = lon
    data['time_zone'] = time_zone

    # === 1.10 Reordenar columnas al formato PVWatts ===
    cols = ['Year','Month','Day','Hour','Minute','GHI','DNI','DHI','Tdry','Tdew','RH','Pres','Wspd','Wdir','Snow Depth','latitude','longitude','time_zone']
    data_final = data[cols]

    # === 1.11 Eliminar filas inválidas (basura) ===
    data_final = data_final[pd.to_numeric(data_final['Year'], errors='coerce').notnull()]

    # === 1.12 Calcular nivel de irradiancia solo en horas de sol
    data_sol = data_final[data_final['GHI'] > 0]
    ghi_promedio_sol = data_sol['GHI'].mean()
    ghi_max_sol = data_sol['GHI'].max()
    ghi_min_sol = data_sol['GHI'].min()

    print(f"   ➡ Irradiancia Promedio (horas de sol): {ghi_promedio_sol:.2f} W/m²")
    print(f"   ➡ Irradiancia Máxima (horas de sol): {ghi_max_sol:.2f} W/m²")
   

    # === 1.13 Definir nombre del archivo final ===
    nombre_base = os.path.splitext(nombre_archivo)[0]
    nombre_final = f"{nombre_base}_pvwatts_final.csv"

    # === 1.14 Guardar el archivo en formato PVWatts v8 ===
    try:
        with open(nombre_final, 'w', encoding='utf-8') as f:
            f.write("latitude,longitude,timezone,elevation\n")
            f.write(f"{lat},{lon},{time_zone},{elev}\n")
            data_final.drop(columns=['latitude', 'longitude', 'time_zone']).to_csv(f, index=False)
        print(f"✅ Archivo generado: {nombre_final}\n")
    except Exception as e:
        print(f"❌ Error al guardar el archivo: {e}")

# === 2. EJEMPLO DE USO ===
# Cambia aquí el nombre del archivo que quieras procesar
# procesar_csv('tmy_-38.320_-71.960_2005_2023.csv')
# procesar_csv('tmy_29.652_91.172_2005_2023.csv')
procesar_csv('tmy_59.567_150.800_2005_2023.csv')


🔵 Procesando archivo: tmy_59.567_150.800_2005_2023.csv
   ➡ Latitud: 59.567, Longitud: 150.8, Elevación: 62.0 m, Zona horaria estimada: UTC+12
   ➡ Ubicación estimada: Rusia (Kamchatka)
   ➡ Irradiancia Promedio (horas de sol): 241.49 W/m²
   ➡ Irradiancia Máxima (horas de sol): 835.95 W/m²
✅ Archivo generado: tmy_59.567_150.800_2005_2023_pvwatts_final.csv

